# Visual perception from a drone on-board camera


## 1- Introduction

In this exercise we are going to implement a color filter to segment an object in an image. To do it, the student needs to have at least the next knowledge:

- Color spaces (RGB, HSV, etc)
- Python programming skills
- Basic understanding of [OpenCV library](http://opencv.org/)

## 2 - Exercise components

### 2.1 Gazebo simulator

Gazebo simulator will be running in the background. The Gazebo world employed for this exercise has two elements: a simulated Parrot AR.Drone and a static box.

The AR.Drone quadrotor will provide images from its vertical camera where the static box will be visualized.

![Gazebo world](images/gazebo_world.png "Gazebo world")

### 2.2 Color Filter component

This component has been developed specifically to carry out this exercise. This component connects to Gazebo to teleoperate the quadrotor and receives images from its camera.

The student has to modify this component and add code to accomplish the exercise. In particular, it is required to modify the ``execute()`` method.

### 2.3 Printer

This class prints an image in a Jupyter Notebook for debugging purposes. It will receive processed images from Color Filter to debug our algorithm.

## 3 - Exercise initialization

To start coding, we need to call ``ColorFilter`` class once. Run this code and wait a few seconds until color filter initialization finishes with an ``OK`` message:

In [ ]:
#! /usr/bin/env python
import numpy as np
import cv2
from color_filter import ColorFilter
from printer import printImage
%matplotlib inline

# Init color filter
cf = ColorFilter()
cf.play()

Now, we have to tell the simulated AR.Drone to take off, running this code:

In [ ]:
# Take off
cf.extra.takeoff()

Once the robot is flying, we can start coding to give intelligence to the robot. We can do it modifying the ``execute()`` method from Color Filter component. This method will be called iteratively about 10 times per second. To understand how it works, we are going to print a message in each iteration:

In [ ]:
# Implement execute method
def execute(self):
    print "Running execute iteration"
      
cf.setExecute(execute)

Stop printing updating the method with an empty code:

In [ ]:
def execute(self):
    pass
      
cf.setExecute(execute)

## 4 - Image manipulation

Color filter receives images from the simulated robot. To obtain these images you can run this code inside the``execute()`` method:

```
image_input = self.camera.getImage()
```

To debug our code and show the output, there are two images that can be visualized:

- We can visualize an RGB image (three channels), employed to show the images received from the simulator or manipulate them. You can set this images with this code:

```
self.set_color_image(image_three_channels)
```

- We can visualize a gray image (one channels), employed to show the color filter result:

```
self.set_filtered_image(image_one_channels)
```

You can recover these images afterwards with these commands:

In [ ]:
# RGB image
imageRGB = cf.get_color_image()

# Gray image
imageGray = cf.get_filtered_image()

## 5 - Programming a color filter segmentation


To accomplish this exercise the student has to implement a color filter that segments a box and detects its position inside the image.

Thus, given an input image like this image:

![Input image](images/input_img.png "Input image")

The expected output would be similar to this image:

![Output image](images/output_img.png "Output image")

To obtain this result, the proposed pipeline is:

1. Smooth image
2. RGB to HSV conversion
3. Color filter
4. Rectangle approximation
5. Box detection

This steps are detailed in the next sections and can be easily conducted using [OpenCV library](https://opencv.org/ "OpenCV")

### 5.1 - Smooth image

Image smoothing is useful to remove noise or imperfections in image. For this exercise, we recommend to use a *Gaussian Filter*, that can be found in OpenCV library as [GaussianBlur](https://docs.opencv.org/2.4/modules/imgproc/doc/filtering.html?highlight=gaussianblur#gaussianblur). The expected result of this filter from the input image shown in previous section is this:

![Smoothed image](images/sm_img.png "Smoothed image")

### 5.2 - RGB to HSV conversion

The images received from Gazebo have an RGB color space. This color space is useful to represent digital images but it is also very sensitive to light changes. Therefore, the next step is to convert our RGB image into a HSV image. We recommend to use the [cvtColor](https://docs.opencv.org/2.4/modules/imgproc/doc/miscellaneous_transformations.html#cvtcolor) function from OpenCV.

If we print this image, the expected result will be similar to this image:

![HSV image](images/hsv_img.png "HSV image")

### 5.3 - Color filter

Now we can apply our color filter to the HSV image. The OpenCV function [inRange](https://docs.opencv.org/2.4/modules/core/doc/operations_on_arrays.html?highlight=inrange#inrange) can help us to make this color filter.

This function receives two arrays, the first one sets the lower HSV values of the filter, and the second one the upper values. For the first parameter (H), the expected values are in range [0, 180], whereas for S and V the values are in range [0, 255]. Thus, a filter where all pixels would be validated would have this appearance:

```
lower_values = np.array([0,0,0], dtype=np.uint8)
upper_values = np.array([180,255,255], dtype=np.uint8)
```

Note: To represent arrays we employ the [numpy library](http://www.numpy.org/)

Once the maximum and minimum values for each HSV parameters have been properly set, the thresholded image (with one channel) has to be similar to this one:

![Thresholded image](images/thr_img.png "Thresholded image")

### 5.4 - Rectangle approximation

Trying to find a white object inside a black background is easier than trying to find a colored box inside a changing background. Therefor, from now on, we will use the thresholded image calculated in the previous step.

One option to detect the box could be detecting the object contour with [findContours](https://docs.opencv.org/2.4/modules/imgproc/doc/structural_analysis_and_shape_descriptors.html?highlight=findcontours#findcontours) function. This function modifies the input image, so we recommend to create a copy before using it with the [numpy copy](https://docs.scipy.org/doc/numpy/reference/generated/numpy.copy.html) function.

FindContours returns a list of points that defines the object contour. This points can be approximated to polygons using one of the next OpenCV functions: [approxPolyDP](https://docs.opencv.org/2.4/modules/imgproc/doc/structural_analysis_and_shape_descriptors.html?highlight=findcontours#approxpolydp) or [boundingRect](https://docs.opencv.org/2.4/modules/imgproc/doc/structural_analysis_and_shape_descriptors.html?highlight=findcontours#boundingrect). You can also draw the obtained rectangle with [rectangle](https://docs.opencv.org/2.4/modules/core/doc/drawing_functions.html?highlight=rectangle#rectangle) function.

### 5.5 - Box detection

Since color filtering parameters are not easy to adjust (even in simulated environments), there may be several image regions with rectangles calculated in previous step. One rectangle will belong to the box we are trying to detect, where as the rest will be noisy regions.

In this case, we recommend to filter the calculated rectangles to show only the one belonging to the box. You can pick up the proper rectangle setting up some restrictions, like rectangle size or shape.

The final result will be the output image we show at the beginning of this section.

![Output image](images/output_img.png "Output image")

## 6 - Algorithm skeleton

We provide an skeleton where you can code your color filtering following the previous steps:

In [ ]:
def execute(self):
      
    # Get image
    input_image = self.camera.getImage()
    if input_image is None:
        print "Can't get images from camera, is simulator running?"
        return
    
    if input_image.any(): 
        output_img = np.copy(input_image)
        
        # Smooth image
        # Add your code here
        
        # RGB to HSV conversion
        # Add your code here
        
        # Color filter
        # Add your code here
        
        # Rectangle approximation
        # Add your code here
        
        # Box detection
        # Add your code here

        # Save images
        self.set_color_image(output_img)
        #self.set_filtered_image(thresold_img)

cf.setExecute(execute)

You can see saved images running this code:

In [ ]:
# Show color image
imageCamera = cf.get_color_image()
printImage(imageCamera)

In [ ]:
# Show filtered image
imageCamera = cf.get_filtered_image()
printImage(imageCamera)

You can also move the drone to check if you can follow the box in the images:

In [ ]:
cf.move()